## Input Variables

In [27]:
# Input Variables
Brand_name = "Ponds"
TOM_tvc = 60
TOM_TVC_ICA = 72
Spont_Brand_tvc = 75  # Same as BR Unaided - TVC
Spont_Brand_tvc_ica = 80  # Same as BR Unaided - TVC+ICA
Aided_Brand_tvc = 94
Aided_Brand_tvc_ica = 92 
Creative_type = 'F(TVC) + F(ICA)'


## Load and Clean Data

In [28]:
import pandas as pd

# Replace 'Global Campaign Tracker.xlsx' with the path to your file
file_path = "Global Campaign Tracker_Updated.xlsx"

# Load the specific sheet named 'India' using the 'openpyxl' engine
try:
    campaign_data_india = pd.read_excel(file_path, sheet_name='INDIA', engine='openpyxl')
    print("Data from 'India' Sheet Loaded Successfully!")
    print("Columns in Dataset:", campaign_data_india.columns)
    print("First Few Rows:")
    print(campaign_data_india.head())
except Exception as e:
    print(f"Error loading data from 'India' sheet: {e}")


Data from 'India' Sheet Loaded Successfully!
Columns in Dataset: Index(['Year', 'SECTOR', 'CATEGORY', 'UNILEVER CATEGORY', 'ADVERTISER',
       'ADVERTISER GROUP', 'BRAND', 'TARGET AUDIENCE', 'MARKET', 'Product',
       'Activity Period', 'RESEARCH METHODOLOGY', 'CAMPAIGN FORMAT', 'SOURCE',
       'TOTAL ICA SPOTS', 'Channel', 'PROGRAM', 'TVC ARTWORK LINK',
       'Type of TVC (F/E/M)', 'APPROVED MOCKUP/ ICA CREATIVE LINK',
       'Type of ICA (F/E/M)', 'CAMPAIGN TYPE/ OBJECTIVE', 'TOM-ICA',
       'TOM - Control', 'TOM Uplift (Control vs ICA)', 'TOM - TVC',
       'TOM - TVC+ICA', 'TOM Uplift (TVC vs TVC + ICA)', 'BR Unaided ICA',
       'BR Unaided Control', 'BR Unaided Uplift (Control vs ICA)',
       'BR Unaided - TVC', 'BR Unaided - TVC+ICA',
       'BR Unaided Uplift (TVC vs TVC + ICA)', 'AD RECALL - ICA',
       'AD RECALL - CONTROL', 'AD RECALL Uplift (ICA-CONTROL)',
       'AD RECALL - TVC', 'AD RECALL - TVC+ICA',
       ' AD RECALL Uplift (TVC vs TVC +ICA)', 'MR - ICA', 'MR -

In [29]:
# List of columns to keep; Remove unused columns for managability
columns_to_keep = [
    'Year', 'SECTOR', 'CATEGORY', 'ADVERTISER', 'BRAND', 'TARGET AUDIENCE',
    'MARKET', 'CAMPAIGN FORMAT', 'TOM - TVC', 'TOM - TVC+ICA', 
    'TOM Uplift (TVC vs TVC + ICA)', 'BR Unaided - TVC', 
    'BR Unaided - TVC+ICA', 'BR Unaided Uplift (TVC vs TVC + ICA)', 
    'Type of TVC (F/E/M)', 'Type of ICA (F/E/M)', 'MR - TVC', 'MR - TVC+ICA',
    'MR Uplift (TVC vs TVC + ICA)'
]

# Filter the data to include only the specified columns
campaign_data_india = campaign_data_india[columns_to_keep]

# Display the first few rows of the filtered data to verify
print(campaign_data_india.head())

     Year                                            SECTOR  \
0  2021.0                                  FOOD & BEVERAGES   
1  2021.0                                  FOOD & BEVERAGES   
2  2021.0  BUILDING, INDUSTRIAL & LAND MATERIALS/EQUIPMENTS   
3  2021.0                                  FOOD & BEVERAGES   
4  2021.0                              PERSONAL ACCESSORIES   

                     CATEGORY                   ADVERTISER  \
0                  CHOCOLATES        HERSHEY INDIA PVT LTD   
1                         TEA             A V THOMAS GROUP   
2  BUILDING MATERIALS/SYSTEMS          PIDILITE INDUSTRIES   
3      SOFT DRINK NON AERATED          COCA COLA INDIA LTD   
4    RETAIL OUTLETS-JEWELLERS  ALUKKAS ENTERPRISES PVT LTD   

                             BRAND   TARGET AUDIENCE      MARKET  \
0       HERSHEYS KISSES CHOCOLATES      AA 15-40 A U   Karnataka   
1                 AVT GOLD CUP TEA  AA 25-50 ABC U+R  Tamil Nadu   
2  ROFF TILE & STONE FIXING EXPERT        M 

In [30]:
# Calculate the Spont Brand Uplift percentage for each record and add it as a new column
campaign_data_india['Spont Brand Uplift (%)'] = (
    (campaign_data_india['BR Unaided - TVC+ICA'] - campaign_data_india['BR Unaided - TVC']) /
    campaign_data_india['BR Unaided - TVC']
) * 100

print(campaign_data_india.head())

     Year                                            SECTOR  \
0  2021.0                                  FOOD & BEVERAGES   
1  2021.0                                  FOOD & BEVERAGES   
2  2021.0  BUILDING, INDUSTRIAL & LAND MATERIALS/EQUIPMENTS   
3  2021.0                                  FOOD & BEVERAGES   
4  2021.0                              PERSONAL ACCESSORIES   

                     CATEGORY                   ADVERTISER  \
0                  CHOCOLATES        HERSHEY INDIA PVT LTD   
1                         TEA             A V THOMAS GROUP   
2  BUILDING MATERIALS/SYSTEMS          PIDILITE INDUSTRIES   
3      SOFT DRINK NON AERATED          COCA COLA INDIA LTD   
4    RETAIL OUTLETS-JEWELLERS  ALUKKAS ENTERPRISES PVT LTD   

                             BRAND   TARGET AUDIENCE      MARKET  \
0       HERSHEYS KISSES CHOCOLATES      AA 15-40 A U   Karnataka   
1                 AVT GOLD CUP TEA  AA 25-50 ABC U+R  Tamil Nadu   
2  ROFF TILE & STONE FIXING EXPERT        M 

In [31]:
# Get the total number of rows and columns
total_rows, total_columns = campaign_data_india.shape

# Print the total number of rows
print(f"Total number of rows: {total_rows}")

# Check for missing values
print(f"\nTotal null values in each row: \n{campaign_data_india.isnull().sum()}")

# If we want to drop rows with missing values or fill them with a default value
# campaign_data_india = campaign_data_india.dropna()  # or
# campaign_data_india = campaign_data_india.fillna(0)


Total number of rows: 1052

Total null values in each row: 
Year                                    625
SECTOR                                  666
CATEGORY                                666
ADVERTISER                              666
BRAND                                   666
TARGET AUDIENCE                         667
MARKET                                  667
CAMPAIGN FORMAT                         671
TOM - TVC                               767
TOM - TVC+ICA                           767
TOM Uplift (TVC vs TVC + ICA)           767
BR Unaided - TVC                        767
BR Unaided - TVC+ICA                    767
BR Unaided Uplift (TVC vs TVC + ICA)    767
Type of TVC (F/E/M)                     848
Type of ICA (F/E/M)                     782
MR - TVC                                774
MR - TVC+ICA                            774
MR Uplift (TVC vs TVC + ICA)            774
Spont Brand Uplift (%)                  767
dtype: int64


In [32]:
print(campaign_data_india.dtypes)  # Check for correct data types


Year                                    float64
SECTOR                                   object
CATEGORY                                 object
ADVERTISER                               object
BRAND                                    object
TARGET AUDIENCE                          object
MARKET                                   object
CAMPAIGN FORMAT                          object
TOM - TVC                               float64
TOM - TVC+ICA                           float64
TOM Uplift (TVC vs TVC + ICA)           float64
BR Unaided - TVC                        float64
BR Unaided - TVC+ICA                    float64
BR Unaided Uplift (TVC vs TVC + ICA)    float64
Type of TVC (F/E/M)                      object
Type of ICA (F/E/M)                      object
MR - TVC                                float64
MR - TVC+ICA                            float64
MR Uplift (TVC vs TVC + ICA)            float64
Spont Brand Uplift (%)                  float64
dtype: object


In [33]:
print(campaign_data_india.describe())


              Year   TOM - TVC  TOM - TVC+ICA  TOM Uplift (TVC vs TVC + ICA)  \
count   427.000000  285.000000     285.000000                     285.000000   
mean   2022.576112   44.277193      60.771930                      16.624561   
std       1.169122   23.091909      22.892598                       9.400110   
min    2021.000000    1.000000       3.000000                      -1.000000   
25%    2021.000000   27.000000      48.000000                      10.000000   
50%    2023.000000   45.000000      65.000000                      16.000000   
75%    2024.000000   61.000000      77.000000                      22.000000   
max    2024.000000   89.000000      97.000000                      54.000000   

       BR Unaided - TVC  BR Unaided - TVC+ICA  \
count        285.000000            285.000000   
mean          62.715789             76.722807   
std           22.205029             19.895245   
min            1.000000             10.000000   
25%           50.000000           

In [34]:
print(campaign_data_india['SECTOR'].value_counts())
print()
print(campaign_data_india['TARGET AUDIENCE'].value_counts())


SECTOR
FOOD & BEVERAGES                                    130
BUILDING, INDUSTRIAL & LAND MATERIALS/EQUIPMENTS     39
SERVICES                                             30
PERSONAL CARE/PERSONAL HYGIENE                       23
BANKING/FINANCE/INVESTMENT                           20
DURABLES                                             20
TELECOM PRODUCTS                                     17
LAUNDRY                                              15
HOUSEHOLD PRODUCTS                                   14
COMPUTERS                                            14
RETAIL                                               12
PERSONAL HEALTHCARE                                  12
AUTO                                                 12
HAIR CARE                                            10
PERSONAL ACCESSORIES                                  6
COSMETICS                                             4
FUEL/PETROLEUM PRODUCTS                               2
BABYCARE                                 

In [35]:
#remove rows that do not have TVC-ICA scores
filtered_data = campaign_data_india.dropna(subset=['BR Unaided - TVC', 'BR Unaided - TVC+ICA'])

#Filter to include only records where 'BR Unaided - TVC' is greater than 25
filtered_data = filtered_data[filtered_data['BR Unaided - TVC'] > 25]

print(filtered_data.describe())

              Year   TOM - TVC  TOM - TVC+ICA  TOM Uplift (TVC vs TVC + ICA)  \
count   261.000000  261.000000     261.000000                     261.000000   
mean   2022.509579   47.574713      64.425287                      16.992337   
std       1.168869   21.222792      19.811981                       9.014515   
min    2021.000000    1.000000       5.000000                      -1.000000   
25%    2021.000000   33.000000      53.000000                      10.000000   
50%    2023.000000   48.000000      68.000000                      17.000000   
75%    2024.000000   62.000000      79.000000                      23.000000   
max    2024.000000   89.000000      97.000000                      50.000000   

       BR Unaided - TVC  BR Unaided - TVC+ICA  \
count        261.000000            261.000000   
mean          67.283525             81.302682   
std           16.891397             12.493858   
min           26.000000             48.000000   
25%           55.000000           

In [36]:
# # Ensure that values in target audience column are treated as strings
# filtered_data['TARGET AUDIENCE'] = filtered_data['TARGET AUDIENCE'].astype(str)
# # Filter rows where Target Audience is Female
filtered_data = filtered_data[filtered_data['TARGET AUDIENCE'].str[0] == 'F']

sector = 'FOOD & BEVERAGES'

filtered_data = filtered_data[filtered_data['SECTOR'] == sector]

print(filtered_data.head())

# # Display the first few rows of the filtered data for verification
# print(filtered_data.describe())


      Year            SECTOR             CATEGORY  \
8   2021.0  FOOD & BEVERAGES  FOOD PRODUCTS RANGE   
13  2021.0  FOOD & BEVERAGES        NOODLES/PASTA   
25  2021.0  FOOD & BEVERAGES    READY TO EAT FOOD   
27  2021.0  FOOD & BEVERAGES  FOOD PRODUCTS RANGE   
41  2022.0  FOOD & BEVERAGES               SPICES   

                       ADVERTISER                                  BRAND  \
8   WEIKFIELD PRODUCTS CO (I) LTD                WEIKFIELD FOOD PRODUCTS   
13                        ITC LTD                SUNFEAST YIPPEE NOODLES   
25  WEIKFIELD PRODUCTS CO (I) LTD  WEIKFIELD CUSTARD READY TO EAT VANILA   
27   MOTHER DAIRY FRUIT & VEG LTD          MOTHER DAIRY RANGE OF PRODUCT   
41   DHARAMPAL SATYAPAL FOODS LTD                             CATCH HING   

    TARGET AUDIENCE MARKET CAMPAIGN FORMAT  TOM - TVC  TOM - TVC+ICA  \
8      F 22-40 AB U    HSM  TVC vs TVC+ICA        1.0            7.0   
13    F 2-14 AB U+R    HSM  TVC vs TVC+ICA       49.0           53.0   
25     F

In [ ]:
# using BR Unaided - TVC to define the thresholds for small, medium, and large brands.
br_unaided_percentiles = filtered_data['BR Unaided - TVC'].quantile([0.40, 0.69])
print("Percentiles for BR Unaided - TVC:")
print(br_unaided_percentiles)


In [ ]:
def categorize_brand_size(br_unaided_score):
    if br_unaided_score <= br_unaided_percentiles[0.40]:
        return 'Small'
    elif br_unaided_score <= br_unaided_percentiles[0.69]:
        return 'Medium'
    else:
        return 'Large'

filtered_data['Brand Size'] = filtered_data['BR Unaided - TVC'].apply(categorize_brand_size)
print(filtered_data[['BR Unaided - TVC', 'Brand Size']].head())

In [ ]:
# Calculate average TOM uplift and counts for each brand size
tom_uplift_stats = filtered_data.groupby('Brand Size')['TOM Uplift (TVC vs TVC + ICA)'].agg(['mean', 'size'])
print("TOM Uplift Stats by Brand Size:")
print(tom_uplift_stats)

In [52]:
## Visualize the comparison of average uplifts across brand sizes.
# import matplotlib.pyplot as plt

# average_uplifts.plot(kind='bar')
# plt.title('Average Uplift by Brand Size')
# plt.ylabel('Average Uplift (%)')
# plt.xlabel('Brand Size')
# plt.show()

In [ ]:
# Input Variables
Brand_name = "Ponds"
TOM_tvc = 60
TOM_TVC_ICA = 72
Spont_Brand_tvc = 75  # Same as BR Unaided - TVC
Spont_Brand_tvc_ica = 80  # Same as BR Unaided - TVC+ICA

# Calculate the TOM uplift for the current brand
current_tom_uplift = (TOM_TVC_ICA - TOM_tvc) / TOM_tvc * 100  # Percentage uplift

# Determine the current brand size based on Spont_Brand_tvc
def categorize_brand_size(br_unaided_score):
    if br_unaided_score <= br_unaided_percentiles[0.4]:
        return 'Small'
    elif br_unaided_score <= br_unaided_percentiles[0.69]:
        return 'Medium'
    else:
        return 'Large'

current_brand_size = categorize_brand_size(Spont_Brand_tvc)

# Retrieve the average uplift for the current brand size
average_tom_uplift_for_size = tom_uplift_stats['mean'][current_brand_size]

# Compare the current brand uplift to the average
print(f"Brand Name: {Brand_name}")
print(f"Brand Size: {current_brand_size}")
print(f"Current Brand TOM Uplift: {current_tom_uplift:.2f}%")
print(f"Average TOM Uplift for {current_brand_size} Brands: {average_tom_uplift_for_size:.2f}%")

if current_tom_uplift > average_tom_uplift_for_size:
    print(f"The current ad for {Brand_name} shows a **significant improvement** compared to the average uplift for {current_brand_size} brands.")
else:
    print(f"The current ad for {Brand_name} does **not show a significant improvement** compared to the average uplift for {current_brand_size} brands.")


In [ ]:
# Calculate the average Spont Brand uplift percentage for each brand size
average_spont_brand_uplifts = filtered_data.groupby('Brand Size')['Spont Brand Uplift (%)'].mean()
count_spont_brand_uplifts = filtered_data.groupby('Brand Size')['Spont Brand Uplift (%)'].size()

# Print the averages to verify
print("Average Spont Brand Uplift by Brand Size:")
print(average_spont_brand_uplifts)
print(count_spont_brand_uplifts)


## Calculate Average Spont Brand Uplift for Current Brand and Compare to Average

In [ ]:
# Calculate the Spont Brand uplift for the current brand
current_spont_brand_uplift = (Spont_Brand_tvc_ica - Spont_Brand_tvc) / Spont_Brand_tvc * 100  # Percentage uplift

# Retrieve the average Spont Brand uplift for the current brand size
average_spont_uplift_for_size = average_spont_brand_uplifts[current_brand_size]

# Compare the current brand's Spont Brand uplift to the average
print(f"\n--- Spont Brand (BR Unaided) Comparison ---")
print(f"Current Brand Spont Brand Uplift: {current_spont_brand_uplift:.2f}%")
print(f"Average Spont Brand Uplift for {current_brand_size} Brands: {average_spont_uplift_for_size:.2f}%")

if current_spont_brand_uplift > average_spont_uplift_for_size:
    print(f"The current ad for {Brand_name} shows a **significant improvement** in Spont Brand uplift compared to the average for {current_brand_size} brands.")
else:
    print(f"The current ad for {Brand_name} does **not show a significant improvement** in Spont Brand uplift compared to the average for {current_brand_size} brands.")


# Type of TVC vs Type of ICA calculations

In [56]:
# Filter out rows where 'Type of TVC' or 'Type of ICA' are null
filtered_type_data = filtered_data.dropna(subset=['Type of TVC (F/E/M)', 'Type of ICA (F/E/M)'])

# # Display the first few rows to verify
# print("Filtered Data (Non-null Type Columns):")
# print(filtered_type_data.head())

In [ ]:
# Further filter where 'Type of ICA' is 'F'
filtered_ica_f_data = filtered_type_data[filtered_type_data['Type of ICA (F/E/M)'] == 'F']

# Display the first few rows to verify
print(filtered_ica_f_data['Type of ICA (F/E/M)'].value_counts())


In [ ]:
# Step 1: Filter data to include only the specified combinations and target audience starting with 'F'
filtered_combinations_data = filtered_type_data[
    (((filtered_type_data['Type of TVC (F/E/M)'] == 'E') & (filtered_type_data['Type of ICA (F/E/M)'] == 'F')) |
     ((filtered_type_data['Type of TVC (F/E/M)'] == 'F') & (filtered_type_data['Type of ICA (F/E/M)'] == 'F')) |
     ((filtered_type_data['Type of TVC (F/E/M)'] == 'M') & (filtered_type_data['Type of ICA (F/E/M)'] == 'F'))) &
    (filtered_type_data['TARGET AUDIENCE'].str[0] == 'F')  # Additional condition for target audience
]

# Step 2: Define the combinations and calculate metrics
combinations = {
    "E(TVC) + F(ICA)": {'TVC': 'E', 'ICA': 'F'},
    "F(TVC) + F(ICA)": {'TVC': 'F', 'ICA': 'F'},
    "M(TVC) + F(ICA)": {'TVC': 'M', 'ICA': 'F'}
}

# Initialize a dictionary to store results
combination_metrics = {}

# Loop through each combination
for combo_name, combo_values in combinations.items():
    # Filter the data for the current combination
    combo_data = filtered_combinations_data[
        (filtered_combinations_data['Type of TVC (F/E/M)'] == combo_values['TVC']) &
        (filtered_combinations_data['Type of ICA (F/E/M)'] == combo_values['ICA'])
    ]
    
    # Calculate metrics using the new percentage column
    avg_spont_brand_uplift = combo_data['Spont Brand Uplift (%)'].mean()  # Average percentage uplift
    record_count = combo_data.shape[0]  # Number of records
    
    # Store the results
    combination_metrics[combo_name] = {
        "Average Spont Brand Uplift (%)": avg_spont_brand_uplift,
        "Record Count": record_count
    }

# Step 3: Print the results
print("Metrics for Each Combination (Target Audience: Female):")
for combo, metrics in combination_metrics.items():
    print(f"\nCombination: {combo}")
    print(f"Average Spont Brand Uplift (%): {metrics['Average Spont Brand Uplift (%)']:.2f}")
    print(f"Record Count: {metrics['Record Count']}")


In [ ]:
# Retrieve the average uplift for the current creative type
average_uplift_for_current_type = combination_metrics[Creative_type]["Average Spont Brand Uplift (%)"]

# Compare the current ad's uplift to the average
print(f"\n--- Comparison for Creative Type: {Creative_type} ---")
print(f"Current Ad Spont Brand Uplift: {current_spont_brand_uplift:.2f}%")
print(f"Average Spont Brand Uplift for {Creative_type}: {average_uplift_for_current_type:.2f}%")

if current_spont_brand_uplift > average_uplift_for_current_type:
    print(f"The current ad shows a **significant improvement** compared to the average for the same creative type.")
else:
    print(f"The current ad does **not show a significant improvement** compared to the average for the same creative type.")
